In [1]:
import os
from vedo import *
import vtk
from vmtk import pypes
import traceback
import pickle
import networkx as nx
from auxiliares import read_vtk, save_r_p, read_obj
from parseObj import calcularMatriz, calcularMatrizSplines
import Arbol as modelo
from ResamplingRDP import interpolarRDP_conRadio, vtpToObj
from splines import limpiarRadiosSplines, binarizar, traversefeaturesSerializado, calculate_splines, binarizar, grafo2arbol
from collections import Counter
from vtk.util.numpy_support import vtk_to_numpy
import numpy as np
from scipy.interpolate import splprep, splev, make_splprep, BSpline
import matplotlib.pyplot as plt

Obtain centerline

In [2]:
files = os.listdir('../Intra/mallasNormalized')

In [3]:
#files = os.listdir('../models/vessels/remeshed/area-005')

cfolder = os.listdir('../Intra/centerlines')
for file in files:
    if file.split(".")[1] == "vtp":
        if file.split(".")[0] + '-network.vtp' not in cfolder:
            print("Processing file:", file)
            script = 'vmtknetworkextraction '
            input_file =  '-ifile ../Intra/mallasNormalized/' + file 
            output_file = ' -ofile ../Intra/centerlines/' + file.split(".")[0] + '-network.vtp'
            myPype = pypes.PypeRun(script+input_file+output_file)

In [4]:
centerlines = os.listdir("../Intra/centerlines/")
reader = vtk.vtkXMLPolyDataReader()
writer = vtk.vtkXMLPolyDataWriter()
for file in centerlines:
    if file not in os.listdir("../Intra/centerlinesResampled/") and file.split(".")[1] == "vtp":
        print("calculando: ", file)
        reader.SetFileName("../Intra/centerlines/" + file)
        reader.Update()
        # Get the polydata
        centerline = reader.GetOutput()
        print("Centerline points:", centerline)
        resampleada = interpolarRDP_conRadio(centerline, 0.02)
        writer.SetFileName("../Intra/centerlinesResampled/" + file)
        writer.SetInputData(resampleada)
        writer.Write()

centerline to obj

In [5]:
centerfolder = os.listdir("../Intra/centerlinesResampled/")
objfolder = os.listdir("../Intra/centerlinesResampledOBJ/")
#arrays_to_remove = ['EdgeArray', 'EdgePCoordArray', 'TCoords_']
for file in centerfolder:
    if file.split(".")[1] == "vtp" and (file.split(".")[0] + '.obj') not in objfolder:
        print("converting", file)
        centerline = read_vtk("../centerlinesResampled/" + file)            
        vtpToObj(file, "../Intra/centerlinesResampled","../Intra/centerlinesResampledOBJ")

Mesh vtp to obj

In [6]:
objMfolder = os.listdir("../Intra/mallasOBJ")
files = os.listdir("../Intra/mallasNormalized")
#arrays_to_remove = ['EdgeArray', 'EdgePCoordArray', 'TCoords_']
for file in files:
    if file.split(".")[1] == "vtp" and (file.split(".")[0] + '.obj') not in objMfolder:
        print("converting")
        vtpToObj(file, "../Intra/mallasNormalized","../Intra/mallasOBJ")

create radius array

In [7]:
# Input and output directories
input_dir = "../Intra/centerlinesResampled"
output_dir = "../Intra/radius_arrays"
os.makedirs(output_dir, exist_ok=True)

# VTP reader
reader = vtk.vtkXMLPolyDataReader()

# Iterate through all .vtp files in the folder
for filename in os.listdir(input_dir):
    if filename.endswith(".vtp"):
        filepath = os.path.join(input_dir, filename)

        # Read the VTP file
        reader.SetFileName(filepath)
        reader.Update()
        polydata = reader.GetOutput()

        # Get the Radius array from PointData
        point_data = polydata.GetPointData()
        radius_array = point_data.GetArray("Radius")

        if radius_array is not None:
            radius_np = vtk_to_numpy(radius_array)

            # Save to .npy file
            output_filename = os.path.splitext(filename)[0] + "_radius.npy"
            output_path = os.path.join(output_dir, output_filename)
            np.save(output_path, radius_np)
            print(f"Saved: {output_path}")
        else:
            print(f"No 'Radius' array found in {filename}")

Saved: ../Intra/radius_arrays\ArteryObjAN1-0-network_radius.npy
Saved: ../Intra/radius_arrays\ArteryObjAN1-11-network_radius.npy
Saved: ../Intra/radius_arrays\ArteryObjAN1-12-network_radius.npy
Saved: ../Intra/radius_arrays\ArteryObjAN1-15-network_radius.npy
Saved: ../Intra/radius_arrays\ArteryObjAN1-16-network_radius.npy
Saved: ../Intra/radius_arrays\ArteryObjAN1-17-network_radius.npy
Saved: ../Intra/radius_arrays\ArteryObjAN1-19-network_radius.npy
Saved: ../Intra/radius_arrays\ArteryObjAN1-2-network_radius.npy
Saved: ../Intra/radius_arrays\ArteryObjAN1-4-network_radius.npy
Saved: ../Intra/radius_arrays\ArteryObjAN1-5-network_radius.npy
Saved: ../Intra/radius_arrays\ArteryObjAN1-6-network_radius.npy
Saved: ../Intra/radius_arrays\ArteryObjAN1-7-network_radius.npy
Saved: ../Intra/radius_arrays\ArteryObjAN11-0-network_radius.npy
Saved: ../Intra/radius_arrays\ArteryObjAN11-1-network_radius.npy
Saved: ../Intra/radius_arrays\ArteryObjAN11-11-network_radius.npy
Saved: ../Intra/radius_arrays\

create splines

In [8]:
centerfolder = "../Intra/centerlinesResampled/"
meshfolder = "../Intra/mallasNormalized/"
coef_folder = "../Intra/splines/coeficientes/"


centerlines = os.listdir(centerfolder)
centerlines = sorted(centerlines)  
centerlines = centerlines
meshes = os.listdir(meshfolder)

meshes = sorted(meshes)
meshes = meshes 

for mesh in meshes:
    try:
        calculate_splines(mesh, coef_folder, centerfolder, meshfolder)
    except Exception as e:
        traceback.print_exc()
        print(f"Error processing mesh {mesh}")
        

c:\Users\lab03\miniconda3\envs\vm\Lib\site-packages\scipy\interpolate\_fitpack_py.py:159: RuntimeWarning: The required storage space exceeds the available storage space.
Probable causes: data (x,y) size is too small or smoothing parameter
s is too small (fp>s).
  res = _impl.splprep(x, w, u, ub, ue, k, task, s, t, full_output, nest, per,
Traceback (most recent call last):
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in calculate_splines
    ar = [areas[i] for i in res[point]]
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in <listcomp>
    ar = [areas[i] for i in res[point]]
          ~~~~~^^^
IndexError: list index out of range
Traceback (most recent call last):
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in calculate_splines
    ar = [areas[i] for i in res[point]]
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File

EXCEPT
EXCEPT
EXCEPT
EXCEPT


Traceback (most recent call last):
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in calculate_splines
    ar = [areas[i] for i in res[point]]
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in <listcomp>
    ar = [areas[i] for i in res[point]]
          ~~~~~^^^
IndexError: list index out of range
Traceback (most recent call last):
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in calculate_splines
    ar = [areas[i] for i in res[point]]
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in <listcomp>
    ar = [areas[i] for i in res[point]]
          ~~~~~^^^
IndexError: list index out of range


EXCEPT
EXCEPT
EXCEPT


Traceback (most recent call last):
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in calculate_splines
    ar = [areas[i] for i in res[point]]
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in <listcomp>
    ar = [areas[i] for i in res[point]]
          ~~~~~^^^
IndexError: list index out of range
Traceback (most recent call last):
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in calculate_splines
    ar = [areas[i] for i in res[point]]
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in <listcomp>
    ar = [areas[i] for i in res[point]]
          ~~~~~^^^
IndexError: list index out of range
Traceback (most recent call last):
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in calculate_splines

...

Traceback (most recent call last):
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in calculate_splines
    ar = [areas[i] for i in res[point]]
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in <listcomp>
    ar = [areas[i] for i in res[point]]
          ~~~~~^^^
IndexError: list index out of range
Traceback (most recent call last):
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in calculate_splines
    ar = [areas[i] for i in res[point]]
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in <listcomp>
    ar = [areas[i] for i in res[point]]
          ~~~~~^^^
IndexError: list index out of range


EXCEPT
EXCEPT


Traceback (most recent call last):
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in calculate_splines
    ar = [areas[i] for i in res[point]]
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in <listcomp>
    ar = [areas[i] for i in res[point]]
          ~~~~~^^^
IndexError: list index out of range
Traceback (most recent call last):
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in calculate_splines
    ar = [areas[i] for i in res[point]]
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in <listcomp>
    ar = [areas[i] for i in res[point]]
          ~~~~~^^^
IndexError: list index out of range


EXCEPT


Traceback (most recent call last):
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in calculate_splines
    ar = [areas[i] for i in res[point]]
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 301, in <listcomp>
    ar = [areas[i] for i in res[point]]
          ~~~~~^^^
IndexError: list index out of range


create graph radio

In [10]:
gfolder = os.listdir('../Intra/grafos')
l_error = []
meshOBJ = os.listdir("../Intra/mallasOBJ")

for file in meshOBJ:
    print(file)
    if file.split(".")[1] is not None:
        if file.split(".")[1] == "obj":
            fileObj = open("../Intra/centerlinesResampledOBJ/" +file.split(".")[0]+"-network.obj")
            if file.split(".")[0] + '-grafo.gpickle' not in gfolder:
                try: 
                    grafo = calcularMatriz(fileObj, "../Intra/radius_arrays/" + file.split(".")[0] + "-network_radius.npy")
                    print("calculating: ", file)
                    with open("../Intra/grafos/" + file.split(".")[0] + '-grafo.gpickle', 'wb') as f:
                        pickle.dump(grafo, f, pickle.HIGHEST_PROTOCOL)
                except Exception:
                    print("problem with: ", file)
                    l_error.append(file)
                    traceback.print_exc()
gfolder = os.listdir('../Intra/grafos')   
#print("problem with files: ", l_error)

ArteryObjAN1-0.mtl
ArteryObjAN1-0.obj
calculating:  ArteryObjAN1-0.obj
ArteryObjAN1-11.mtl
ArteryObjAN1-11.obj
calculating:  ArteryObjAN1-11.obj
ArteryObjAN1-12.mtl
ArteryObjAN1-12.obj
calculating:  ArteryObjAN1-12.obj
ArteryObjAN1-15.mtl
ArteryObjAN1-15.obj
calculating:  ArteryObjAN1-15.obj
ArteryObjAN1-16.mtl
ArteryObjAN1-16.obj
calculating:  ArteryObjAN1-16.obj
ArteryObjAN1-17.mtl
ArteryObjAN1-17.obj
calculating:  ArteryObjAN1-17.obj
ArteryObjAN1-19.mtl
ArteryObjAN1-19.obj
calculating:  ArteryObjAN1-19.obj
ArteryObjAN1-2.mtl
ArteryObjAN1-2.obj
calculating:  ArteryObjAN1-2.obj
ArteryObjAN1-4.mtl
ArteryObjAN1-4.obj
calculating:  ArteryObjAN1-4.obj
ArteryObjAN1-5.mtl
ArteryObjAN1-5.obj
calculating:  ArteryObjAN1-5.obj
ArteryObjAN1-6.mtl
ArteryObjAN1-6.obj
calculating:  ArteryObjAN1-6.obj
ArteryObjAN1-7.mtl
ArteryObjAN1-7.obj
calculating:  ArteryObjAN1-7.obj
ArteryObjAN11-0.mtl
ArteryObjAN11-0.obj
calculating:  ArteryObjAN11-0.obj
ArteryObjAN11-1.mtl
ArteryObjAN11-1.obj
calculating:  Ar

Traceback (most recent call last):
  File "C:\Users\lab03\AppData\Local\Temp\ipykernel_20756\4126485880.py", line 12, in <module>
    grafo = calcularMatriz(fileObj, "../Intra/radius_arrays/" + file.split(".")[0] + "-network_radius.npy")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 97, in calcularMatriz
    raise Exception("El grafo no se pudo unificar con r=" + str(r) + ". Se tienen " + str(nx.number_connected_components( grafo )) + " componentes conexas.")
Exception: El grafo no se pudo unificar con r=1e-08. Se tienen 2 componentes conexas.


calculating:  ArteryObjAN134-7.obj
ArteryObjAN135-0.mtl
ArteryObjAN135-0.obj
calculating:  ArteryObjAN135-0.obj
ArteryObjAN135-1.mtl
ArteryObjAN135-1.obj
calculating:  ArteryObjAN135-1.obj
ArteryObjAN135-10.mtl
ArteryObjAN135-10.obj
calculating:  ArteryObjAN135-10.obj
ArteryObjAN135-12.mtl
ArteryObjAN135-12.obj
calculating:  ArteryObjAN135-12.obj
ArteryObjAN135-15.mtl
ArteryObjAN135-15.obj
calculating:  ArteryObjAN135-15.obj
ArteryObjAN135-17.mtl
ArteryObjAN135-17.obj
calculating:  ArteryObjAN135-17.obj
ArteryObjAN135-18.mtl
ArteryObjAN135-18.obj
calculating:  ArteryObjAN135-18.obj
ArteryObjAN135-19.mtl
ArteryObjAN135-19.obj
calculating:  ArteryObjAN135-19.obj
ArteryObjAN135-2.mtl
ArteryObjAN135-2.obj
calculating:  ArteryObjAN135-2.obj
ArteryObjAN135-5.mtl
ArteryObjAN135-5.obj
calculating:  ArteryObjAN135-5.obj
ArteryObjAN135-6.mtl
ArteryObjAN135-6.obj
calculating:  ArteryObjAN135-6.obj
ArteryObjAN135-7.mtl
ArteryObjAN135-7.obj
calculating:  ArteryObjAN135-7.obj
ArteryObjAN135-8.mtl
Ar

Traceback (most recent call last):
  File "C:\Users\lab03\AppData\Local\Temp\ipykernel_20756\4126485880.py", line 12, in <module>
    grafo = calcularMatriz(fileObj, "../Intra/radius_arrays/" + file.split(".")[0] + "-network_radius.npy")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 97, in calcularMatriz
    raise Exception("El grafo no se pudo unificar con r=" + str(r) + ". Se tienen " + str(nx.number_connected_components( grafo )) + " componentes conexas.")
Exception: El grafo no se pudo unificar con r=1e-08. Se tienen 2 componentes conexas.
Traceback (most recent call last):
  File "C:\Users\lab03\AppData\Local\Temp\ipykernel_20756\4126485880.py", line 12, in <module>
    grafo = calcularMatriz(fileObj, "../Intra/radius_arrays/" + file.split(".")[0] + "-network_radius.npy")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

calculating:  ArteryObjAN138-8.obj
ArteryObjAN139-0.mtl
ArteryObjAN139-0.obj
calculating:  ArteryObjAN139-0.obj
ArteryObjAN139-1.mtl
ArteryObjAN139-1.obj
calculating:  ArteryObjAN139-1.obj
ArteryObjAN139-11.mtl
ArteryObjAN139-11.obj
calculating:  ArteryObjAN139-11.obj
ArteryObjAN139-12.mtl
ArteryObjAN139-12.obj
calculating:  ArteryObjAN139-12.obj
ArteryObjAN139-13.mtl
ArteryObjAN139-13.obj
calculating:  ArteryObjAN139-13.obj
ArteryObjAN139-14.mtl
ArteryObjAN139-14.obj
calculating:  ArteryObjAN139-14.obj
ArteryObjAN139-15.mtl
ArteryObjAN139-15.obj
calculating:  ArteryObjAN139-15.obj
ArteryObjAN139-16.mtl
ArteryObjAN139-16.obj
calculating:  ArteryObjAN139-16.obj
ArteryObjAN139-17.mtl
ArteryObjAN139-17.obj
calculating:  ArteryObjAN139-17.obj
ArteryObjAN139-2.mtl
ArteryObjAN139-2.obj
calculating:  ArteryObjAN139-2.obj
ArteryObjAN139-3.mtl
ArteryObjAN139-3.obj
calculating:  ArteryObjAN139-3.obj
ArteryObjAN139-4.mtl
ArteryObjAN139-4.obj
calculating:  ArteryObjAN139-4.obj
ArteryObjAN139-6.mtl

...

calculating:  ArteryObjAN25-18.obj
ArteryObjAN25-19.mtl
ArteryObjAN25-19.obj
calculating:  ArteryObjAN25-19.obj
ArteryObjAN25-2.mtl
ArteryObjAN25-2.obj
calculating:  ArteryObjAN25-2.obj
ArteryObjAN25-3.mtl
ArteryObjAN25-3.obj
calculating:  ArteryObjAN25-3.obj
ArteryObjAN25-6.mtl
ArteryObjAN25-6.obj
calculating:  ArteryObjAN25-6.obj
ArteryObjAN25-7.mtl
ArteryObjAN25-7.obj
calculating:  ArteryObjAN25-7.obj
ArteryObjAN25-8.mtl
ArteryObjAN25-8.obj
calculating:  ArteryObjAN25-8.obj
ArteryObjAN26-0.mtl
ArteryObjAN26-0.obj
calculating:  ArteryObjAN26-0.obj
ArteryObjAN26-1.mtl
ArteryObjAN26-1.obj
calculating:  ArteryObjAN26-1.obj
ArteryObjAN26-10.mtl
ArteryObjAN26-10.obj
calculating:  ArteryObjAN26-10.obj
ArteryObjAN26-12.mtl
ArteryObjAN26-12.obj
calculating:  ArteryObjAN26-12.obj
ArteryObjAN26-13.mtl
ArteryObjAN26-13.obj
calculating:  ArteryObjAN26-13.obj
ArteryObjAN26-14.mtl
ArteryObjAN26-14.obj
calculating:  ArteryObjAN26-14.obj
ArteryObjAN26-16.mtl
ArteryObjAN26-16.obj
calculating:  Artery

Traceback (most recent call last):
  File "C:\Users\lab03\AppData\Local\Temp\ipykernel_20756\4126485880.py", line 12, in <module>
    grafo = calcularMatriz(fileObj, "../Intra/radius_arrays/" + file.split(".")[0] + "-network_radius.npy")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 97, in calcularMatriz
    raise Exception("El grafo no se pudo unificar con r=" + str(r) + ". Se tienen " + str(nx.number_connected_components( grafo )) + " componentes conexas.")
Exception: El grafo no se pudo unificar con r=1e-08. Se tienen 2 componentes conexas.
Traceback (most recent call last):
  File "C:\Users\lab03\AppData\Local\Temp\ipykernel_20756\4126485880.py", line 12, in <module>
    grafo = calcularMatriz(fileObj, "../Intra/radius_arrays/" + file.split(".")[0] + "-network_radius.npy")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

calculating:  ArteryObjAN32-19.obj
ArteryObjAN32-5.mtl
ArteryObjAN32-5.obj
problem with:  ArteryObjAN32-5.obj
ArteryObjAN32-6.mtl
ArteryObjAN32-6.obj
calculating:  ArteryObjAN32-6.obj
ArteryObjAN32-9.mtl
ArteryObjAN32-9.obj
calculating:  ArteryObjAN32-9.obj
ArteryObjAN34-0.mtl
ArteryObjAN34-0.obj
calculating:  ArteryObjAN34-0.obj
ArteryObjAN34-11.mtl
ArteryObjAN34-11.obj
calculating:  ArteryObjAN34-11.obj
ArteryObjAN34-12.mtl
ArteryObjAN34-12.obj
calculating:  ArteryObjAN34-12.obj
ArteryObjAN34-15.mtl
ArteryObjAN34-15.obj
calculating:  ArteryObjAN34-15.obj
ArteryObjAN34-16.mtl
ArteryObjAN34-16.obj
calculating:  ArteryObjAN34-16.obj
ArteryObjAN34-18.mtl
ArteryObjAN34-18.obj
calculating:  ArteryObjAN34-18.obj
ArteryObjAN34-19.mtl
ArteryObjAN34-19.obj
calculating:  ArteryObjAN34-19.obj
ArteryObjAN34-2.mtl
ArteryObjAN34-2.obj
calculating:  ArteryObjAN34-2.obj
ArteryObjAN34-3.mtl
ArteryObjAN34-3.obj
calculating:  ArteryObjAN34-3.obj
ArteryObjAN34-4.mtl
ArteryObjAN34-4.obj
calculating:  Arte

Traceback (most recent call last):
  File "C:\Users\lab03\AppData\Local\Temp\ipykernel_20756\4126485880.py", line 12, in <module>
    grafo = calcularMatriz(fileObj, "../Intra/radius_arrays/" + file.split(".")[0] + "-network_radius.npy")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 97, in calcularMatriz
    raise Exception("El grafo no se pudo unificar con r=" + str(r) + ". Se tienen " + str(nx.number_connected_components( grafo )) + " componentes conexas.")
Exception: El grafo no se pudo unificar con r=1e-08. Se tienen 2 componentes conexas.


calculating:  ArteryObjAN44-6.obj
ArteryObjAN44-8.mtl
ArteryObjAN44-8.obj
calculating:  ArteryObjAN44-8.obj
ArteryObjAN44-9.mtl
ArteryObjAN44-9.obj
calculating:  ArteryObjAN44-9.obj
ArteryObjAN54-12.mtl
ArteryObjAN54-12.obj
calculating:  ArteryObjAN54-12.obj
ArteryObjAN54-4.mtl
ArteryObjAN54-4.obj
calculating:  ArteryObjAN54-4.obj
ArteryObjAN54-5.mtl
ArteryObjAN54-5.obj
calculating:  ArteryObjAN54-5.obj
ArteryObjAN55-1.mtl
ArteryObjAN55-1.obj
calculating:  ArteryObjAN55-1.obj
ArteryObjAN55-10.mtl
ArteryObjAN55-10.obj
calculating:  ArteryObjAN55-10.obj
ArteryObjAN55-11.mtl
ArteryObjAN55-11.obj
calculating:  ArteryObjAN55-11.obj
ArteryObjAN55-12.mtl
ArteryObjAN55-12.obj
calculating:  ArteryObjAN55-12.obj
ArteryObjAN55-13.mtl
ArteryObjAN55-13.obj
calculating:  ArteryObjAN55-13.obj
ArteryObjAN55-14.mtl
ArteryObjAN55-14.obj
calculating:  ArteryObjAN55-14.obj
ArteryObjAN55-15.mtl
ArteryObjAN55-15.obj
calculating:  ArteryObjAN55-15.obj
ArteryObjAN55-16.mtl
ArteryObjAN55-16.obj
calculating:  A

create binary tree radio

In [11]:
t_list = os.listdir('../Intra/TreesNumpy/')
gfolder = os.listdir('../Intra/grafos')   
print("Grafo files: ", gfolder)
print("TreesNumpy files: ", t_list)
files = os.listdir('../Intra/mallasOBJ/')
for file in files:
    
    if file.split(".")[0] + '-grafo.gpickle' in gfolder and file.split(".")[1] == 'obj' and (file.split(".")[0] + '.npy') not in t_list:

        try:
            grafo = pickle.load(open('../Intra/grafos/' + file.split(".")[0] + '-grafo.gpickle',  'rb'))
            grafo = grafo.to_undirected()
            nc = 0
            #controlo si tiene ciclos
            if len(nx.cycle_basis(grafo))>0:
                nc = 1
            

            nb = 0
            #for nodo in grafo.nodes:
            for nodo in grafo.nodes:
                if len(grafo.edges(nodo))>3:
                    #print("binario", len(grafo.edges(nodo)))
                    binarizar(grafo)
                    #nb = 1
                    break
            
            if nb ==0 and nc == 0: 
                aRecorrer = []
                numeroNodoInicial = 1
                distancias = nx.floyd_warshall( grafo )

                parMaximo = (-1, -1)
                maxima = -1
            
                for nodoInicial in distancias.keys():
                    for nodoFinal in distancias[nodoInicial]:
                        if distancias[nodoInicial][nodoFinal] > maxima:
                            maxima = distancias[nodoInicial] [nodoFinal]
                            parMaximo = (nodoInicial, nodoFinal)

                for nodo in grafo.nodes:
                    if distancias[parMaximo[0]][nodo] == int( maxima / 2):
                        numeroNodoInicial = nodo
                        if len(grafo.edges(numeroNodoInicial))>2:
                            numeroNodoInicial = list(grafo.edges(numeroNodoInicial))[0][1]

                        break
            
        
                rad = list(grafo.nodes[numeroNodoInicial]['radio'])
                nodoRaiz = modelo.Node( numeroNodoInicial, radius =  rad )

                for vecino in grafo.neighbors( numeroNodoInicial ):
                    if vecino != numeroNodoInicial:
                        aRecorrer.append( (vecino, numeroNodoInicial,nodoRaiz ) )
                
                while len(aRecorrer) != 0:
                    nodoAAgregar, numeroNodoPadre,nodoPadre = aRecorrer.pop(0)
                    radius = list(grafo.nodes[nodoAAgregar]['radio'])
                    nodoActual = modelo.Node( nodoAAgregar, radius =  radius)
                    nodoPadre.agregarHijo( nodoActual )
                    for vecino in grafo.neighbors( nodoAAgregar ):
                        if vecino != numeroNodoPadre:
                            aRecorrer.append( (vecino, nodoAAgregar,nodoActual) )

    
                serial = nodoRaiz.serialize(nodoRaiz)
        
                f = []
                traversefeaturesSerializado(nodoRaiz, f, k=4)
            
                try:
                    array = np.array(f)
                    ##GUARDO
                    np.save("../Intra/TreesNumpy/" + file.split(".")[0], array)
                    #print("flat", array)
                    print(f"calculated {file}")
                    print("number of nodes: ", array.shape)
                except Exception: 
                    
                    traceback.print_exc()
                    print(f"error with file{file}")

            else:
                print("error with file: ", file, nb, nc)
        except:
            print("error with file: ", file, nb, nc)

Grafo files:  ['ArteryObjAN1-0-grafo.gpickle', 'ArteryObjAN1-11-grafo.gpickle', 'ArteryObjAN1-12-grafo.gpickle', 'ArteryObjAN1-15-grafo.gpickle', 'ArteryObjAN1-16-grafo.gpickle', 'ArteryObjAN1-17-grafo.gpickle', 'ArteryObjAN1-19-grafo.gpickle', 'ArteryObjAN1-2-grafo.gpickle', 'ArteryObjAN1-4-grafo.gpickle', 'ArteryObjAN1-5-grafo.gpickle', 'ArteryObjAN1-6-grafo.gpickle', 'ArteryObjAN1-7-grafo.gpickle', 'ArteryObjAN11-0-grafo.gpickle', 'ArteryObjAN11-1-grafo.gpickle', 'ArteryObjAN11-11-grafo.gpickle', 'ArteryObjAN11-16-grafo.gpickle', 'ArteryObjAN11-17-grafo.gpickle', 'ArteryObjAN11-2-grafo.gpickle', 'ArteryObjAN11-3-grafo.gpickle', 'ArteryObjAN11-4-grafo.gpickle', 'ArteryObjAN11-5-grafo.gpickle', 'ArteryObjAN11-6-grafo.gpickle', 'ArteryObjAN11-8-grafo.gpickle', 'ArteryObjAN116-0-grafo.gpickle', 'ArteryObjAN116-1-grafo.gpickle', 'ArteryObjAN116-10-grafo.gpickle', 'ArteryObjAN116-11-grafo.gpickle', 'ArteryObjAN116-12-grafo.gpickle', 'ArteryObjAN116-15-grafo.gpickle', 'ArteryObjAN116-16-gr

In [12]:
t_list = os.listdir('../Intra/Trees')
objfolder = os.listdir('../Intra/mallasOBJ')
for file in objfolder:
    if file.split(".")[0] + '-grafo.gpickle' in gfolder and file.split(".")[1] == 'obj' and (file.split(".")[0] + '_tree.dat') not in t_list:
        
        grafo = pickle.load(open('../Intra/grafos/' + file.split(".")[0] + '-grafo.gpickle',  'rb'))
        grafo = grafo.to_undirected()

        nc = 0
        #controlo si tiene ciclos
        if len(nx.cycle_basis(grafo))>0:
            #print("ciclos", len(nx.cycle_basis(grafo)))
            nc = 1
        
        nb = 0
        #controlo si es binario (no hay nodos con mas de 3 edges)
        for nodo in grafo.nodes:
            if len(grafo.edges(nodo))>3:
                #print("binario", len(grafo.edges(nodo)))
                binarizar(grafo)
                #nb = 1
                break
        
        if nb ==0 and nc == 0: 
            try:
                serial = grafo2arbol(grafo)
                #write serialized string to file
                print("calculated file: ", file)
                file = open("../Intra/Trees/" + file.split(".")[0] +"_tree.dat", "w")
                file.write(serial)
                file.close()
            except:
                print("error with file: ", file)
                traceback.print_exc()
        else:
            print("Not binary or with cycles: ", file)

calculated file:  ArteryObjAN1-0.obj
calculated file:  ArteryObjAN1-11.obj
calculated file:  ArteryObjAN1-12.obj
calculated file:  ArteryObjAN1-15.obj
calculated file:  ArteryObjAN1-16.obj
calculated file:  ArteryObjAN1-17.obj
calculated file:  ArteryObjAN1-19.obj
calculated file:  ArteryObjAN1-2.obj
calculated file:  ArteryObjAN1-4.obj
calculated file:  ArteryObjAN1-5.obj
calculated file:  ArteryObjAN1-6.obj
calculated file:  ArteryObjAN1-7.obj
calculated file:  ArteryObjAN11-0.obj
calculated file:  ArteryObjAN11-1.obj
calculated file:  ArteryObjAN11-11.obj
calculated file:  ArteryObjAN11-16.obj
calculated file:  ArteryObjAN11-17.obj
calculated file:  ArteryObjAN11-2.obj
calculated file:  ArteryObjAN11-3.obj
calculated file:  ArteryObjAN11-4.obj
calculated file:  ArteryObjAN11-5.obj
calculated file:  ArteryObjAN11-6.obj
calculated file:  ArteryObjAN11-8.obj
calculated file:  ArteryObjAN116-0.obj
calculated file:  ArteryObjAN116-1.obj
calculated file:  ArteryObjAN116-10.obj
calculated 

creo grafo con splines

In [13]:
gfolder = sorted(os.listdir('../Intra/grafosSplines'))
files = sorted(os.listdir('../Intra/mallasOBJ'))

l_error = []
for file in files:
   
    if file.split(".")[1] is not None:
        if file.split(".")[1] == "obj":
            try:
                fileObj = open("../Intra/centerlinesResampledOBJ/"+file.split(".")[0]+"-network.obj")
            except Exception:
                print("1 problem with: ", file)
                l_error.append(file)
                #traceback.print_exc()
            if file.split(".")[0] + '-grafo.gpickle' not in gfolder:
                try: 
                    grafo = calcularMatrizSplines(fileObj, "../Intra/splines/coeficientes/" + file.split(".")[0] + ".pkl")
                    print("calculating: ", file)
                    with open("../Intra/grafosSplines/" + file.split(".")[0] + '-grafo.gpickle', 'wb') as f:
                        pickle.dump(grafo, f, pickle.HIGHEST_PROTOCOL)
                except Exception:
                    print("2 problem with: ", file)
                    l_error.append(file)
                    traceback.print_exc()

gfolder = os.listdir('../Intra/grafosSplines')  
#print("problem with files: ", l_error)

calculating:  ArteryObjAN1-0.obj
calculating:  ArteryObjAN1-11.obj
calculating:  ArteryObjAN1-12.obj
calculating:  ArteryObjAN1-15.obj
calculating:  ArteryObjAN1-16.obj
calculating:  ArteryObjAN1-17.obj
calculating:  ArteryObjAN1-19.obj
calculating:  ArteryObjAN1-2.obj
calculating:  ArteryObjAN1-4.obj
calculating:  ArteryObjAN1-5.obj
calculating:  ArteryObjAN1-6.obj
calculating:  ArteryObjAN1-7.obj
calculating:  ArteryObjAN11-0.obj
calculating:  ArteryObjAN11-1.obj
calculating:  ArteryObjAN11-11.obj
calculating:  ArteryObjAN11-16.obj
calculating:  ArteryObjAN11-17.obj
calculating:  ArteryObjAN11-2.obj
2 problem with:  ArteryObjAN11-3.obj
calculating:  ArteryObjAN11-4.obj
2 problem with:  ArteryObjAN11-5.obj
calculating:  ArteryObjAN11-6.obj
calculating:  ArteryObjAN11-8.obj
calculating:  ArteryObjAN116-0.obj
calculating:  ArteryObjAN116-1.obj
calculating:  ArteryObjAN116-10.obj


Traceback (most recent call last):
  File "C:\Users\lab03\AppData\Local\Temp\ipykernel_20756\2936710654.py", line 17, in <module>
    grafo = calcularMatrizSplines(fileObj, "../Intra/splines/coeficientes/" + file.split(".")[0] + ".pkl")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 103, in calcularMatrizSplines
    grafo, arbolVertices = calcularGrafoYArbolSplines( fileObj, fileRadios )
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 50, in calcularGrafoYArbolSplines
    vertices.append( (len(verticesCrudos), {'radio': (vertice[0], vertice[1], vertice[2],radios[len(verticesCrudos)], knots[len(verticesCrudos)])} ))
                                                                                         ~~~~~~^^^^^^^^^^^^^^^^^^^

calculating:  ArteryObjAN116-11.obj
calculating:  ArteryObjAN116-12.obj
calculating:  ArteryObjAN116-15.obj
calculating:  ArteryObjAN116-16.obj
2 problem with:  ArteryObjAN116-17.obj
calculating:  ArteryObjAN116-2.obj
calculating:  ArteryObjAN116-3.obj
calculating:  ArteryObjAN116-4.obj
calculating:  ArteryObjAN116-6.obj
calculating:  ArteryObjAN116-7.obj
2 problem with:  ArteryObjAN116-8.obj
calculating:  ArteryObjAN116-9.obj


Traceback (most recent call last):
  File "C:\Users\lab03\AppData\Local\Temp\ipykernel_20756\2936710654.py", line 17, in <module>
    grafo = calcularMatrizSplines(fileObj, "../Intra/splines/coeficientes/" + file.split(".")[0] + ".pkl")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 103, in calcularMatrizSplines
    grafo, arbolVertices = calcularGrafoYArbolSplines( fileObj, fileRadios )
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 50, in calcularGrafoYArbolSplines
    vertices.append( (len(verticesCrudos), {'radio': (vertice[0], vertice[1], vertice[2],radios[len(verticesCrudos)], knots[len(verticesCrudos)])} ))
                                                                                         ~~~~~~^^^^^^^^^^^^^^^^^^^

calculating:  ArteryObjAN117-0.obj
calculating:  ArteryObjAN117-10.obj
calculating:  ArteryObjAN117-11.obj
calculating:  ArteryObjAN117-12.obj
calculating:  ArteryObjAN117-14.obj
calculating:  ArteryObjAN117-15.obj
calculating:  ArteryObjAN117-16.obj
2 problem with:  ArteryObjAN117-18.obj
calculating:  ArteryObjAN117-19.obj
calculating:  ArteryObjAN117-2.obj
calculating:  ArteryObjAN117-3.obj
calculating:  ArteryObjAN117-5.obj
calculating:  ArteryObjAN117-6.obj
calculating:  ArteryObjAN117-7.obj
calculating:  ArteryObjAN117-8.obj
calculating:  ArteryObjAN117-9.obj
calculating:  ArteryObjAN119-0.obj
calculating:  ArteryObjAN119-1.obj
2 problem with:  ArteryObjAN119-10.obj
calculating:  ArteryObjAN119-11.obj
calculating:  ArteryObjAN119-12.obj
calculating:  ArteryObjAN119-13.obj
calculating:  ArteryObjAN119-14.obj
calculating:  ArteryObjAN119-15.obj
calculating:  ArteryObjAN119-16.obj
calculating:  ArteryObjAN119-17.obj


...

calculating:  ArteryObjAN44-9.obj
calculating:  ArteryObjAN54-12.obj
calculating:  ArteryObjAN54-4.obj
calculating:  ArteryObjAN54-5.obj
2 problem with:  ArteryObjAN55-1.obj
calculating:  ArteryObjAN55-10.obj
calculating:  ArteryObjAN55-11.obj
calculating:  ArteryObjAN55-12.obj
calculating:  ArteryObjAN55-13.obj
calculating:  ArteryObjAN55-14.obj
calculating:  ArteryObjAN55-15.obj
2 problem with:  ArteryObjAN55-16.obj


Traceback (most recent call last):
  File "C:\Users\lab03\AppData\Local\Temp\ipykernel_20756\2936710654.py", line 17, in <module>
    grafo = calcularMatrizSplines(fileObj, "../Intra/splines/coeficientes/" + file.split(".")[0] + ".pkl")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 103, in calcularMatrizSplines
    grafo, arbolVertices = calcularGrafoYArbolSplines( fileObj, fileRadios )
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 50, in calcularGrafoYArbolSplines
    vertices.append( (len(verticesCrudos), {'radio': (vertice[0], vertice[1], vertice[2],radios[len(verticesCrudos)], knots[len(verticesCrudos)])} ))
                                                                                         ~~~~~~^^^^^^^^^^^^^^^^^^^

calculating:  ArteryObjAN55-18.obj
calculating:  ArteryObjAN55-19.obj
calculating:  ArteryObjAN55-2.obj
calculating:  ArteryObjAN55-3.obj
calculating:  ArteryObjAN55-4.obj
calculating:  ArteryObjAN55-5.obj
calculating:  ArteryObjAN55-6.obj
calculating:  ArteryObjAN55-8.obj
calculating:  ArteryObjAN55-9.obj
calculating:  ArteryObjAN58-0.obj
calculating:  ArteryObjAN58-1.obj
calculating:  ArteryObjAN58-10.obj
calculating:  ArteryObjAN58-11.obj
2 problem with:  ArteryObjAN58-12.obj
calculating:  ArteryObjAN58-14.obj
calculating:  ArteryObjAN58-15.obj
calculating:  ArteryObjAN58-16.obj
calculating:  ArteryObjAN58-17.obj
calculating:  ArteryObjAN58-19.obj
calculating:  ArteryObjAN58-2.obj
calculating:  ArteryObjAN58-3.obj
2 problem with:  ArteryObjAN58-4.obj
calculating:  ArteryObjAN58-5.obj
2 problem with:  ArteryObjAN58-6.obj
calculating:  ArteryObjAN58-7.obj


Traceback (most recent call last):
  File "C:\Users\lab03\AppData\Local\Temp\ipykernel_20756\2936710654.py", line 17, in <module>
    grafo = calcularMatrizSplines(fileObj, "../Intra/splines/coeficientes/" + file.split(".")[0] + ".pkl")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 103, in calcularMatrizSplines
    grafo, arbolVertices = calcularGrafoYArbolSplines( fileObj, fileRadios )
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 50, in calcularGrafoYArbolSplines
    vertices.append( (len(verticesCrudos), {'radio': (vertice[0], vertice[1], vertice[2],radios[len(verticesCrudos)], knots[len(verticesCrudos)])} ))
                                                                                         ~~~~~~^^^^^^^^^^^^^^^^^^^

calculating:  ArteryObjAN58-8.obj
calculating:  ArteryObjAN6-0.obj
calculating:  ArteryObjAN6-12.obj
calculating:  ArteryObjAN6-14.obj
calculating:  ArteryObjAN6-15.obj
calculating:  ArteryObjAN6-19.obj
calculating:  ArteryObjAN6-2.obj
calculating:  ArteryObjAN6-3.obj
calculating:  ArteryObjAN6-4.obj
calculating:  ArteryObjAN6-7.obj
calculating:  ArteryObjAN6-8.obj
calculating:  ArteryObjAN9-0.obj
calculating:  ArteryObjAN9-12.obj
calculating:  ArteryObjAN9-13.obj
calculating:  ArteryObjAN9-14.obj
calculating:  ArteryObjAN9-18.obj
calculating:  ArteryObjAN9-19.obj
calculating:  ArteryObjAN9-2.obj
calculating:  ArteryObjAN9-3.obj
calculating:  ArteryObjAN9-4.obj
calculating:  ArteryObjAN9-5.obj
calculating:  ArteryObjAN9-6.obj
calculating:  ArteryObjAN9-7.obj


creo arbol con splines

In [14]:
t_list = os.listdir('../Intra/TreesSplines/')


for file in files:
   
    if file.split(".")[0] + '-grafo.gpickle' in gfolder and file.split(".")[1] == 'obj' and (file.split(".")[0] + '.npy') not in t_list:

        try:
            grafo = pickle.load(open('../Intra/grafosSplines/' + file.split(".")[0] + '-grafo.gpickle',  'rb'))
            grafo = grafo.to_undirected()
            nc = 0
            #controlo si tiene ciclos
            if len(nx.cycle_basis(grafo))>0:
                nc = 1
            

            nb = 0
            #for nodo in grafo.nodes:
            for nodo in grafo.nodes:
                if len(grafo.edges(nodo))>3:
                    #print("binario", len(grafo.edges(nodo)))
                    binarizar(grafo)
                    #nb = 1
                    break
            
            if nb ==0 and nc == 0: 
                aRecorrer = []
                numeroNodoInicial = 1
                distancias = nx.floyd_warshall( grafo )

                parMaximo = (-1, -1)
                maxima = -1
            
                for nodoInicial in distancias.keys():
                    for nodoFinal in distancias[nodoInicial]:
                        if distancias[nodoInicial][nodoFinal] > maxima:
                            maxima = distancias[nodoInicial] [nodoFinal]
                            parMaximo = (nodoInicial, nodoFinal)

                for nodo in grafo.nodes:
                    if distancias[parMaximo[0]][nodo] == int( maxima / 2):
                        numeroNodoInicial = nodo
                        if len(grafo.edges(numeroNodoInicial))>2:
                            numeroNodoInicial = list(grafo.edges(numeroNodoInicial))[0][1]

                        break
            
        
                rad = list(grafo.nodes[numeroNodoInicial]['radio'])
                rad = limpiarRadiosSplines(rad)
                nodoRaiz = modelo.Node( numeroNodoInicial, radius =  rad )

                for vecino in grafo.neighbors( numeroNodoInicial ):
                    if vecino != numeroNodoInicial:
                        aRecorrer.append( (vecino, numeroNodoInicial,nodoRaiz ) )
                
                while len(aRecorrer) != 0:
                    nodoAAgregar, numeroNodoPadre,nodoPadre = aRecorrer.pop(0)
                    radius = list(grafo.nodes[nodoAAgregar]['radio'])
                    radius = limpiarRadiosSplines(radius)
                    nodoActual = modelo.Node( nodoAAgregar, radius =  radius)
                    nodoPadre.agregarHijo( nodoActual )
                    for vecino in grafo.neighbors( nodoAAgregar ):
                        if vecino != numeroNodoPadre:
                            aRecorrer.append( (vecino, nodoAAgregar,nodoActual) )

    
                serial = nodoRaiz.serialize(nodoRaiz)
        
                f = []
                traversefeaturesSerializado(nodoRaiz, f, k=39)
                print("number of nodes: ", len(f))
               
                
                array = np.array(f)
                ##GUARDO
                np.save("../Intra/TreesSplines/" + file.split(".")[0], array)
                #print("flat", array)
                print(f"calculated {file}")
                
            else:
                print("error with file: ", file, nb, nc)
        except:
            print("error with file: ", file, nb, nc)


number of nodes:  35
calculated ArteryObjAN1-0.obj
number of nodes:  21
calculated ArteryObjAN1-11.obj
number of nodes:  27
calculated ArteryObjAN1-12.obj
number of nodes:  17
calculated ArteryObjAN1-15.obj
number of nodes:  57
calculated ArteryObjAN1-16.obj
number of nodes:  35
calculated ArteryObjAN1-17.obj
number of nodes:  31
calculated ArteryObjAN1-19.obj
number of nodes:  17
calculated ArteryObjAN1-2.obj
number of nodes:  25
calculated ArteryObjAN1-4.obj
number of nodes:  63
calculated ArteryObjAN1-5.obj
number of nodes:  23
calculated ArteryObjAN1-6.obj
number of nodes:  31
calculated ArteryObjAN1-7.obj
number of nodes:  25
calculated ArteryObjAN11-0.obj
number of nodes:  55
calculated ArteryObjAN11-1.obj
number of nodes:  31
calculated ArteryObjAN11-11.obj
number of nodes:  31
calculated ArteryObjAN11-16.obj
number of nodes:  41
calculated ArteryObjAN11-17.obj
number of nodes:  63
calculated ArteryObjAN11-2.obj
number of nodes:  27
calculated ArteryObjAN11-4.obj
number of nodes

In [ ]:
t = np.load("../Intra/TreesSplines/ArteryObjAN1-2.npy")
print(t.shape)
print(t[0])
print(t[-1])

(17, 39)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[-0.52338368 -0.30382839 -0.09188857 -0.62160504 -0.65687454 -0.4001154
 -0.40582183 -0.50820112 -0.62160504 -0.65687454 -0.4001154  -0.12672102
 -0.30354974 -0.52520734 -0.35330325 -0.20398638 -0.12672102 -0.30354974
 -0.52520734 -0.09499199 -0.35881379 -0.08701952  0.08939327  0.04760017
 -0.09499199 -0.35881379 -0.08701952 -0.52811307 -0.38800934 -0.30419114
  0.          0.24636351  0.47188693  0.61199069  0.69580889  1.
  1.24636352  1.47188699  1.61199069]


: 

In [ ]:
t = np.load("../modelsNormalizado/TreesNumpy/C0002.npy")
print(t.shape)
print(t[0])
print(t[-1])

(125, 4)
[0. 0. 0. 0.]
[ 0.20150365 -0.08272542 -0.3654407   0.12599117]


In [ ]:
import re
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Node:
    """
    Class Node
    """
    def __init__(self, value, radius, left = None, right = None):
        self.left = left
        self.data = value
        self.radius = radius
        self.right = right
        

def createNode(data, radius, left = None, right = None, ):
        """
        Utility function to create a node.
        """
        return Node(data, radius, left, right)

def read_tree(filename, dir):
    with open('./' +dir +'/' +filename, "r") as f:
        byte = f.read() 
        return byte

def deserialize(data):
    if  not data:
        return 
    nodes = data.split(';')  
    def post_order(nodes):
        if nodes[-1] == '#':
            nodes.pop()
            return None
        node = nodes.pop().split('_')
        try:
            data = int(node[0])
        except:
            numbers = re.findall(r'\d+', node[0])
            # Convert the extracted numbers to integers
            data = [int(num) for num in numbers]
        radius = node[1]
        rad = radius.split(",")
        rad [0] = rad[0].replace('[','')
        rad [3] = rad[3].replace(']','')
        r = []
        for value in rad:
            r.append(float(value))
        r = torch.tensor(r, device=device)
        root = createNode(data, r)
        root.right = post_order(nodes)
        root.left = post_order(nodes)
        
        return root    
    return post_order(nodes) 


def numerar_nodos(root, count):
    if root is not None:
        numerar_nodos(root.left, count)
        root.data = len(count)
        count.append(1)
        numerar_nodos(root.right, count)
        return 

tree = read_tree('C0002_tree.dat', '../modelsNormalizado/Trees')
s = tree.split(';')
print("Number of nodes in the tree string:", len(s))
deserial = deserialize(tree)
c = []
numerar_nodos(deserial, c)
print(len(c))

Number of nodes in the tree string: 125
62
